This experiment, higher learning rate. Larger rollouts.

In [1]:
import deep_rl

In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
import torch
from torch.nn import functional as F
from torch.autograd import Variable
from torch import nn, optim
import torch.utils.data

# load as dask array
import time

import logging
import sys
import os
import glob
import numpy as np
import datetime
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook as tqdm

In [4]:
from deep_rl.utils.logger import get_logger, get_default_log_dir

from deep_rl.network.network_heads import CategoricalActorCriticNet, QuantileNet, OptionCriticNet, DeterministicActorCriticNet, GaussianActorCriticNet
from deep_rl.network.network_bodies import FCBody
from deep_rl.utils.normalizer import RunningStatsNormalizer, RescaleNormalizer
from deep_rl.component.task import ParallelizedTask

In [5]:
from world_models_sonic.models.vae import VAE5, loss_function_vae
from world_models_sonic.helpers.summarize import TorchSummarizeDf
from world_models_sonic.models.rnn import MDNRNN
from world_models_sonic.models.inverse_model import InverseModel, InverseModel2
from world_models_sonic.models.world_model import WorldModel
from world_models_sonic.custom_envs.env import make_env
from world_models_sonic.custom_envs.wrappers import RandomGameReset
from world_models_sonic import config
from world_models_sonic.helpers.deep_rl import PPOAgent, run_iterations, SonicWorldModelDeepRL, CategoricalWorldActorCriticNet, Config

Importing 0 potential games...
Imported 0 games


# Init

In [6]:
cuda = torch.cuda.is_available()
env_name = 'sonic256'
z_dim = 512  # latent dimensions
channels = 3*4

# RNN
action_dim = 10
image_size = 128

verbose = True  # Set this true to render (and make it go slower)

NAME = 'RNN_v3b_128im_512z_1512_v6l_VAE5_curiosity'
ppo_save_file = './outputs/{NAME}/PPO_512z_all_c.pkl'.format(NAME=NAME)

if not os.path.isdir('./outputs/{NAME}'.format(NAME=NAME)):
    os.makedirs('./outputs/{NAME}'.format(NAME=NAME))

# Log to file and stream
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger(NAME)

log_dir = log_dir='./outputs/{NAME}'.format(NAME=NAME)
print(log_dir)

deep_rl_logger = get_logger(
    NAME,
    file_name='deep_rl_ppo.log',
    level=logging.INFO,
    log_dir='./outputs/{NAME}'.format(NAME=NAME), )

./outputs/RNN_v3b_128im_512z_1512_v6l_VAE5_curiosity


# World model

In [7]:
# Load VAE
# TODO swap z and k dim, since it's inconsistent with other models
vae = VAE5(image_size=image_size, z_dim=128, conv_dim=64, code_dim=8, k_dim=z_dim, channels=channels)
    
# Load MDRNN
action_dim, hidden_size, n_mixture, temp = action_dim, z_dim*2, 5, 0.0

mdnrnn = MDNRNN(z_dim, action_dim, hidden_size, n_mixture, temp)
    
finv = InverseModel2(z_dim, action_dim, hidden_size=64)
    
world_model = WorldModel(vae, mdnrnn, finv, logger=deep_rl_logger, lambda_vae_kld=1 / 10., lambda_finv=1/50, lambda_vae=1/8, lambda_loss=1000)
world_model = world_model.train()
if cuda:
    world_model = world_model.cuda()

In [8]:
# Optimiser for world models
import torch.optim.lr_scheduler
torch.cuda.empty_cache()
optimizer = optim.Adam(world_model.parameters(), lr=5e-6)

world_model.optimizer = optimizer

# Train

In [9]:
z_state_dim=world_model.mdnrnn.z_dim + world_model.mdnrnn.hidden_size


def task_fn(log_dir):
    return SonicWorldModelDeepRL(
        env_fn=lambda: RandomGameReset(make_env(
            'sonic', max_episode_steps=1000, to_gray=False, image_size=image_size)),
        log_dir=log_dir,
        verbose=verbose
    )

config = Config()

verbose = True  # Set this true to render (and make it go slower)
config.num_workers = 1 if verbose else 10
config.task_fn = lambda: ParallelizedTask(
    task_fn, config.num_workers, single_process=config.num_workers == 1)
config.optimizer_fn = lambda params: torch.optim.RMSprop(params, 3e-5)
config.network_fn = lambda state_dim, action_dim: CategoricalWorldActorCriticNet(
    state_dim, action_dim, FCBody(z_state_dim, hidden_units=(64, 64), gate=F.relu), gpu=0 if cuda else -1, world_model_fn=lambda: world_model,
    render=(config.num_workers==1 and verbose),
    z_shape=(32, 16)
)
config.discount = 0.99
config.logger = deep_rl_logger
config.use_gae = True
config.gae_tau = 0.95
config.entropy_weight = 0.01
config.gradient_clip = 50
config.rollout_length = 1*20
config.optimization_epochs = 3
config.num_mini_batches = 4
config.ppo_ratio_clip = 0.1
config.iteration_log_interval = 20

config.train_world_model = True
config.world_model_batch_size = 1

# I tuned these so the intrinsic reward was 1) within an order of magnitude of the extrinsic. 2) smaller, 3) negative when stuck
# TODO use reward normalisers etc to reduce the need for these hyperparameters
config.curiosity = True
config.curiosity_only = True
config.curiosity_weight = 0.000005
config.curiosity_boredom = 1 # how many standard deviations above the mean does it's new experience need to be, so it's not bored
config.reward_normalizer = RunningStatsNormalizer()
config.intrinsic_reward_normalizer = RunningStatsNormalizer()
agent = PPOAgent(config)

print('rollout of ', config.rollout_length*config.num_workers)
print('mini batch', (config.rollout_length*config.num_workers)/config.num_mini_batches)
print('sequence of batch', (config.rollout_length))

if os.path.isfile(ppo_save_file):
    print('loading ppo_save_file', ppo_save_file, 'modified', time.ctime(os.path.getmtime(ppo_save_file)))
    agent.load(ppo_save_file)
    
    # also load normalizers
    state_dict = torch.load(ppo_save_file.replace('.pkl', '-intrinsic_reward_normalizer.pkl'))
    config.intrinsic_reward_normalizer.load_state_dict(state_dict)

    state_dict = torch.load(ppo_save_file.replace('.pkl', '-reward_normalizer.pkl'))
    config.reward_normalizer.load_state_dict(state_dict)
else:
    print("couldn't find save")

game: SonicAndKnuckles3-Genesis state: CarnivalNightZone.Act1
rollout of  20
mini batch 5.0
sequence of batch 20
loading ppo_save_file ./outputs/RNN_v3b_128im_512z_1512_v6l_VAE5_curiosity/PPO_512z_all_c.pkl modified Mon Jun 18 07:06:17 2018


In [10]:
# # # DEBUG

# # # reset from checkpoint
# # agent.load('./outputs/RNN_v3b_128im_512z_1512_v6j_VAE5_all/PPO_512z_all_g-20180606_02-06-59.pkl')

# # # # Reset just rnn
# # world_model.mdnrnn = MDNRNN(z_dim, action_dim, hidden_size, n_mixture, temp)
# # world_model.mdnrnn.cuda()

# world_model.finv = finv = InverseModel2(z_dim, action_dim, hidden_size=64)
# world_model.finv.cuda()

# # # # if we want to reset the actor
# from deep_rl.network.network_heads import ActorCriticNet
# agent.network.network = ActorCriticNet(agent.network.z_state_dim, action_dim, FCBody(z_state_dim, hidden_units=(64, 64), gate=F.relu), None, None)
# agent.network.network.cuda()

In [11]:
print(""""
# To monitor with tensorboard at http://localhost:6006/
cd ./outputs/{NAME}/
tensorboard  --logdir .
""".format(NAME=NAME))

"
# To monitor with tensorboard at http://localhost:6006/
cd ./outputs/RNN_v3b_128im_512z_1512_v6l_VAE5_curiosity/
tensorboard  --logdir .



In [ ]:
try:
    run_iterations(agent, log_dir=log_dir)
except:
    if config.num_workers == 1:
        agent.task.tasks[0].env.close()
    else:
        [t.close() for t in agent.task.tasks]
    print("saving", ppo_save_file)
    agent.save(ppo_save_file)
    torch.save(config.intrinsic_reward_normalizer.state_dict(), ppo_save_file.replace('.pkl', '-intrinsic_reward_normalizer.pkl'))
    torch.save(config.reward_normalizer.state_dict(), ppo_save_file.replace('.pkl', '-reward_normalizer.pkl'))

    # Backup since it sometimes get's corrupted
    ts = datetime.datetime.utcnow().strftime('%Y%m%d_%H')
    print("saving backup",
          ppo_save_file.replace('.pkl', '-%s.pkl' % ts),)
    agent.save(ppo_save_file.replace('.pkl', '-%s.pkl' % ts))
    raise
    

action true/pred (prob): 4 4 (0.9980)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)


2018-06-18 18:11:00,476 - RNN_v3b_128im_512z_1512_v6l_VAE5_curiosity - INFO: rollout extrinsic, intrinsic reward [min/mean/max]: 0.8830/0.8830/0.8830, -0.0000/-0.0000/-0.0000
2018-06-18 18:11:00,605 - RNN_v3b_128im_512z_1512_v6l_VAE5_curiosity - INFO: steps: 20, steps/s: 4.31
  world model losses: 38.9089 rnn=30.4784, inv= 0.0000=0.0200 * 0.0001, vae=8.4306=0.1250 * (59.8973 + 0.1000 * 75.4735)
  epoch reward:       0.0000/0.0000/0.0000 [n=1] (min/mean/max)
  running reward:     0.0000/0.0000/0.0000 [n=1]



action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/

action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/pred (prob): 4 4 (1.0000)
action true/

In [ ]:
agent.save(ppo_save_file)
torch.save(config.intrinsic_reward_normalizer.state_dict(), ppo_save_file.replace('.pkl', '-intrinsic_reward_normalizer.pkl'))
torch.save(config.reward_normalizer.state_dict(), ppo_save_file.replace('.pkl', '-reward_normalizer.pkl'))

# Backup since it sometimes get's corrupted
ts = datetime.datetime.utcnow().strftime('%Y%m%d_%H')
print("saving backup",
      ppo_save_file.replace('.pkl', '-%s.pkl' % ts),)
agent.save(ppo_save_file.replace('.pkl', '-%s.pkl' % ts))

2018-06-16 08:09:05,226 - RNN_v3b_128im_512z_1512_v6k_VAE5_curiosity - INFO: rollout extrinsic, intrinsic reward [min/mean/max]: 0.4335/0.4335/0.4335, 0.0000/0.0872/1.1133
2018-06-16 08:09:05,357 - RNN_v3b_128im_512z_1512_v6k_VAE5_curiosity - INFO: steps: 1640200, steps/s: 26.21
  world model losses: 34.8937 rnn=24.7663, inv= 0.0006=0.0100 * 0.0613, vae=10.1268=0.1250 * (76.6872 + 0.0625 * 69.2362)
  epoch reward:       0.0000/2.5341/24.4756 [n=10] (min/mean/max)
  running reward:     0.0000/2.3456/24.4756 [n=500]

2018-06-12 05:31:16,551 - RNN_v3b_128im_512z_1512_v6k_VAE5_curiosity - INFO: loss_rnn=25.0999, loss_inv= 14.2090=0.0200 * 710.4479, loss_vae=12.6325=0.1250 * (100.8083 + 0.0039 * 64.3391)
2018-06-12 05:31:16,553 - RNN_v3b_128im_512z_1512_v6k_VAE5_curiosity - INFO: total steps 1570752, min/mean/max reward 0.6707/3.2924/10.5535 of 8
2018-06-12 05:31:16,554 - RNN_v3b_128im_512z_1512_v6k_VAE5_curiosity - INFO: running min/mean/max reward 0.6707/5.3926/24.5866 of 500 22.8842 step/s

In [ ]:
agent.save(ppo_save_file)
torch.save(config.intrinsic_reward_normalizer.state_dict(), ppo_save_file.replace('.pkl', '-intrinsic_reward_normalizer.pkl'))
torch.save(config.reward_normalizer.state_dict(), ppo_save_file.replace('.pkl', '-reward_normalizer.pkl'))

In [ ]:
# TODO plot rewards over it

In [ ]:
agent.save(ppo_save_file)
ppo_save_file

# Summarize model

In [ ]:
from IPython.display import display

with torch.no_grad():
    img = np.random.randn(image_size, image_size, 3)
    action = np.array(np.random.randint(0,action_dim))[np.newaxis]
    action = Variable(torch.from_numpy(action)).float().cuda()[np.newaxis]
    gpu_img = Variable(torch.from_numpy(img[np.newaxis].transpose(0, 3, 1, 2))).float().cuda()
    if cuda:
        gpu_img = gpu_img.cuda()
    with TorchSummarizeDf(vae) as tdf:
        x, mu_vae, logvar_vae = vae.forward(gpu_img)
        z = vae.sample(mu_vae, logvar_vae)
        df_vae = tdf.make_df()

    display(df_vae[df_vae.level<2])
    
    with TorchSummarizeDf(mdnrnn) as tdf: 
        pi, mu, sigma, hidden_state = mdnrnn.forward(z.unsqueeze(1).repeat((1,2,1)))
        z_next = mdnrnn.sample(pi, mu, sigma)
        df_mdnrnn = tdf.make_df()
    
    display(df_mdnrnn)
    

    with TorchSummarizeDf(finv) as tdf:
        finv(z.repeat((1,2,1)), z_next)   
        df_finv = tdf.make_df()
    display(df_finv)

    with TorchSummarizeDf(world_model) as tdf:
        world_model(gpu_img, action)
        df_world_model = tdf.make_df()
    display(df_world_model[df_world_model.level<2])
    
    del img, action, gpu_img, x, mu, z, z_next, mu_vae, pi, sigma, logvar_vae